In [73]:
import pandas as pd
from pathlib import Path
from os import listdir
import os.path
from collections import defaultdict
import re
import numpy as np

In [74]:
repo_dir = Path("/home/luk/repos/RAPid-Learn/data")
experiment_id = "a4ea72e0f8064182b7e26b7afe3f5b1e-2021-10-01_08:38:00-policy_gradient-10000episodes-rewardshapingon-egreedy"
experiment_path = repo_dir / experiment_id

In [75]:
novelties = ["axefirecteasy", "prenovelty", "axetobreakeasy", "axetobreakhard", "firecraftingtableeasy",
             "firecraftingtablehard", "rubbertree", "rubbertreehard", "scrapeplank"]
CONVERGENCE_SUCCESS_RATIO = 0.96

In [76]:
def extract_number(f):
    s = re.findall("\d+", f)
    return int(s[0]) if s else -1

In [77]:
csvs_per_novelty = defaultdict(list)
for novelty in novelties:
    for trial_dir in listdir(experiment_path / novelty):
        if os.path.isdir(experiment_path / novelty / trial_dir):
            for filename in os.listdir(experiment_path / novelty / trial_dir):
                if filename.endswith(".csv"):
                    csvs_per_novelty[novelty].append(experiment_path / novelty / trial_dir / filename)
                                       

In [68]:
dfs = []
for novelty in novelties:

    for csv_path in csvs_per_novelty[novelty]:
        
        df = pd.read_csv(csv_path, skiprows=[0])

        df["trial"] = extract_number(str(os.path.basename(os.path.normpath(csv_path))))
        df["novelty"] = novelty
        
        df["ts_trained"] = df["l"].loc[df['mode'] == "learn"].cumsum()
        df["ts_trained"].ffill(inplace=True)
        if novelty == "prenovelty":
            df["episode_counter"] = df.index
            df["episode_counter"].loc[df['mode'] == "eval"] = np.nan
            df["episode_counter"].ffill(inplace=True)
        dfs.append(df)
results_df = pd.concat(dfs).reset_index()

In [69]:
results_df.novelty.unique()

array(['axefirecteasy', 'prenovelty', 'axetobreakeasy', 'axetobreakhard',
       'firecraftingtableeasy', 'firecraftingtablehard', 'rubbertree',
       'rubbertreehard', 'scrapeplank'], dtype=object)

In [70]:
agg_results_df = results_df.loc[results_df['mode'] == "eval"] \
.groupby(by=["novelty", "episode_counter"]) \
.agg({'success': ['mean', 'std'], 'r': ['mean', 'std'], "ts_trained": ['mean', 
                                                                      'std']}).reset_index()

In [72]:
with pd.option_context('display.max_rows', None, 'display.max_columns', None):  # more options can be specified also
    print(agg_results_df)

                   novelty episode_counter success                  r  \
                                              mean       std     mean   
0            axefirecteasy             100    0.00  0.000000  -225.00   
1            axefirecteasy             200    0.10  0.303046   -62.00   
2            axefirecteasy             300    0.00  0.000000  -185.00   
3            axefirecteasy             400    0.00  0.000000  -130.00   
4            axefirecteasy             500    0.00  0.000000  -200.00   
5            axefirecteasy             600    0.00  0.000000  -220.00   
6            axefirecteasy             700    0.00  0.000000  -260.00   
7            axefirecteasy             800    0.00  0.000000  -210.00   
8            axefirecteasy             900    0.00  0.000000  -185.00   
9            axefirecteasy            1000    0.00  0.000000  -205.00   
10           axefirecteasy            1100    0.00  0.000000  -210.00   
11           axefirecteasy            1200    0.00 

In [29]:
results_df.to_csv(repo_dir / experiment_id / "full_results.csv")

In [56]:
agg_results_df.to_csv(repo_dir / experiment_id / "parsed_results.csv")

In [12]:
repo_dir / experiment_id / "parsed_results.csv"

PosixPath('/home/luk/repos/RAPid-Learn/data/a4ea72e0f8064182b7e26b7afe3f5b1e-2021-10-01_08:38:00-policy_gradient-10000episodes-rewardshapingon-egreedy/parsed_results.csv')

In [ ]:
timestep_results_df = results_df.loc[results_df['mode'] == "eval"] \
.groupby(by=["novelty", "episode_counter"]) \
.agg({'success': ['mean', 'std'], 'r': ['mean', 'std'], "ts_trained": ['mean', 
                                                                      'std']}).reset_index()